In [53]:
import pandas as pd
import numpy as np


# Kanton

In [54]:

kanton = './data/plz_verzeichnis.csv'
df_kanton = pd.read_csv(kanton, sep=';')
#df_kanton.columns = df_cntrycd.columns.str.lower()
display(df_kanton.shape)
display(df_kanton.head())

(5307, 17)

,REC_ART,ONRP,BFSNR,PLZ_TYP,POSTLEITZAHL,PLZ_ZZ,GPLZ,ORTBEZ18,ORTBEZ27,KANTON,SPRACHCODE,SPRACHCODE_ ABW,BRIEFZ_DURCH,GILT_AB_DAT,PLZ_BRIEFZUST,PLZ_COFF,Geokoordinaten
0,1,105,5586,80,1000,1,1000,Lausanne 1 Dépôt,Lausanne 1 Dépôt,VD,2,NaN,130,2005-04-01,100060,NaN,NaN
1,1,107,5586,80,1000,3,1000,Lausanne 3,Lausanne 3,VD,2,NaN,130,1986-05-21,100060,NaN,NaN
2,1,110,5586,80,1000,6,1000,Lausanne Ouchy,Lausanne Ouchy,VD,2,NaN,130,1993-02-10,100060,NaN,NaN
3,1,112,5586,80,1000,8,1000,Lausanne Bellevaux,Lausanne Bellevaux,VD,2,NaN,130,1997-05-20,100060,NaN,NaN
4,1,121,5586,80,1000,21,1000,Lausanne Grangette,Lausanne Grangette,VD,2,NaN,130,1986-05-21,100060,NaN,NaN


# SUBSCRIPTIONS

In [55]:
# define the name of the csv file you want to read in
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'

In [56]:
# Read subscriptions
df_subscriptions = pd.read_csv(subscriptions)
df_subscriptions.columns = df_subscriptions.columns.str.lower()

display(df_subscriptions.shape)
display(df_subscriptions.head(10))

(12115, 9)

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription
0,1b82b2308dcae546ad0194d03b9d23edd5d2781109de28...,ab1bcf2c1b8d4eb422079becc0c59b1c7db2fd009235d2...,chf,90.0,BASIC,0,2017-02-24,2021-02-23,False
1,4cb9f67d44d60eca0485270d73b3028143021296081847...,ee1a45a439dc6968f1cfc2b9840264deeada4c0d93f389...,chf,90.0,BASIC,0,2017-08-21,2023-08-20,False
2,40f9e218b70a6f482ecfcde0011305d93fa6059f624a50...,8bf8f58e615549df5f05b717b6e914da80e213edab3dba...,chf,9.0,BASIC,1,2017-09-26,2022-10-08,False
3,69d21e601612cc97fc7df9bd17a281879f785358d1c681...,cf914aa939499e8e18fdd9494fce5297b9dce2e1f909ea...,chf,90.0,BASIC,0,2017-09-28,2021-09-27,False
4,29f98a3154a679f2fe953b1f3b148ddffcb28e60fff719...,8759ab0e44c1cf688770220cf1b87efac2726baca6bfcb...,eur,11.0,STANDARD,1,2017-10-13,2021-10-12,False
5,f5627e4ef66f219cfbcfb507a056bf1a0629f085a46e92...,c5511d59d31159ecb87ca1d7cfb3e44a6c4a4b96e5632a...,chf,240.0,PATRON,0,2017-11-18,2020-11-17,False
6,8e42bc7151b4b0dd29e362b95fec4aa3b8e53c532d40ba...,7f1182ed60acb48e4a55eb54b91bca7f20c55c066d8848...,chf,150.0,STANDARD,0,2017-12-20,2021-12-19,False
7,9f22270608209a507f93f3a74448a78a82f9741bc36ea3...,2c2621bf34f1cc419c0868f27ffece66dc1fb4f5ec665c...,chf,9.0,BASIC,1,2017-12-21,2020-11-08,False
8,03411f3ad7d86f2e1df3db5a337c3025ba35159eb9b41e...,163db260041d30a58540a19312d416c9fc7cc696754108...,chf,9.0,BASIC,1,2017-12-22,2020-10-08,False
9,c254cee83dfc67c794b843a0ad622456cb36fdfe5143c4...,47e795383bff3d37a251be1d911ffc0a7ca6f0da8b70a4...,chf,240.0,PATRON,0,2017-12-31,2022-12-30,False


In [57]:
df_subscriptions.groupby(['currency','price']).subscription_type.value_counts()

currency  price  subscription_type
chf       0.0    PATRON                  7
                 BASIC                   6
                 STANDARD                5
          9.0    BASIC                2917
          15.0   STANDARD             2118
          27.0   BASIC                   1
          67.5   STANDARD                1
          75.0   STANDARD                2
          82.0   BASIC                   1
          90.0   BASIC                2533
                 FULLACCESS              1
                 STANDARD                1
          102.5  PATRON                  2
          150.0  STANDARD             1146
          172.5  PATRON                  1
          192.0  PATRON                  1
          195.0  PATRON                  1
          210.0  PATRON                  1
          220.0  PATRON                  1
          240.0  PATRON               1052
eur       6.0    BASIC                   1
          7.5    BASIC                 272
          11.0   ST

In [58]:
#set data types for subscription dates to datetime
df_subscriptions['subscription_start'] = pd.to_datetime(df_subscriptions['subscription_start'])
df_subscriptions['subscription_end'] = pd.to_datetime(df_subscriptions['subscription_end'])

In [59]:
# Create new 'clean' DataFrame 
df_subscriptions_clean = df_subscriptions

In [60]:
# drop "wrong" subscription type line (FULLACCESS - unknown type to us - only one line therefore decided to drop)
df_subscriptions_clean.drop(df_subscriptions_clean[(df_subscriptions['subscription_type'] == 'FULLACCESS')].index, inplace = True)

In [61]:
#create new column and calculate subscription duration for calculating actual price per subscription
df_subscriptions_clean['subscription_months_raw'] = ((df_subscriptions_clean.subscription_end) - df_subscriptions_clean.subscription_start)/np.timedelta64(1, 'M')

In [62]:
#create new column with rounded subscription months
#generally round up from 0.1 to be able to allow some discrepancies due to day to day calculation of subscription duration (deduct 0.1 to be able to use .ceil)
df_subscriptions_clean['subscription_months'] = df_subscriptions_clean['subscription_months_raw'] - 0.1
df_subscriptions_clean['subscription_months'] = df_subscriptions_clean['subscription_months'].apply(np.ceil)

In [63]:
# Create two columns for chf and eur based on the subscription_type and prices from the filmingo website

# create a list of our conditions
conditions = [
    ((df_subscriptions_clean['subscription_type'] == 'BASIC') & (df_subscriptions_clean['subscription_monthly'] == 0)),
    ((df_subscriptions_clean['subscription_type'] == 'BASIC') & (df_subscriptions_clean['subscription_monthly'] == 1)),
    ((df_subscriptions_clean['subscription_type'] == 'STANDARD') & (df_subscriptions_clean['subscription_monthly'] == 0)),
    ((df_subscriptions_clean['subscription_type'] == 'STANDARD') & (df_subscriptions_clean['subscription_monthly'] == 1)),
    ((df_subscriptions_clean['subscription_type'] == 'PATRON') & (df_subscriptions_clean['subscription_monthly'] == 0))

]

# create a list of the values we want to assign for each condition
values_chf = ['90.0', '9.0', '150.0', '15.0', '240.0']
values_eur = ['75.0', '7.5', '125.0', '12.5', '200.0']

# create a new column and use np.select to assign values to it using our lists as arguments
df_subscriptions_clean['price_chf'] = np.select(conditions, values_chf)
df_subscriptions_clean['price_eur'] = np.select(conditions, values_eur)

#change datatype into float for further calculation
df_subscriptions_clean['price_chf'] = df_subscriptions_clean.price_chf.astype('float')
df_subscriptions_clean['price_eur'] = df_subscriptions_clean.price_eur.astype('float')

# decided to use these prices for all subscriptions regardless if they might have a different prices in the list (possibly due to discounts, total lines of abnormal prices: 39) or are gifted subscription (price: NaN, total lines 1.636)


In [64]:
#calculate total price per subscription (price / 12 * subscription months)
df_subscriptions_clean['total_price_chf'] = df_subscriptions_clean['price_chf'] / 12 * df_subscriptions_clean['subscription_months']
df_subscriptions_clean['total_price_eur'] = df_subscriptions_clean['price_eur'] / 12 * df_subscriptions_clean['subscription_months']

# conditional calculation for exceptions:

# if the subscription is monthly only calculate price * months
df_subscriptions_clean.loc[(df_subscriptions_clean['subscription_monthly'] == 1), 'total_price_chf'] = (df_subscriptions_clean['price_chf'] * df_subscriptions_clean['subscription_months'])
df_subscriptions_clean.loc[(df_subscriptions_clean['subscription_monthly'] == 1), 'total_price_eur'] = (df_subscriptions_clean['price_eur'] * df_subscriptions_clean['subscription_months'])

# if the subscription is gifted and 6 months long, a different price is applicable (there is only a 6 month subscription available for gifted subscriptions)
df_subscriptions_clean.loc[((df_subscriptions_clean['gift_subscription'] == True) & (df_subscriptions_clean['subscription_months'] == 6)), 'total_price_chf'] = '49'
df_subscriptions_clean.loc[((df_subscriptions_clean['gift_subscription'] == True) & (df_subscriptions_clean['subscription_months'] == 6)), 'total_price_eur'] = '41'

In [65]:
df_subscriptions_clean.head()

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription,subscription_months_raw,subscription_months,price_chf,price_eur,total_price_chf,total_price_eur
0,1b82b2308dcae546ad0194d03b9d23edd5d2781109de28...,ab1bcf2c1b8d4eb422079becc0c59b1c7db2fd009235d2...,chf,90.0,BASIC,0,2017-02-24,2021-02-23,False,47.968131,48.0,90.0,75.0,360.0,300.0
1,4cb9f67d44d60eca0485270d73b3028143021296081847...,ee1a45a439dc6968f1cfc2b9840264deeada4c0d93f389...,chf,90.0,BASIC,0,2017-08-21,2023-08-20,False,71.952196,72.0,90.0,75.0,540.0,450.0
2,40f9e218b70a6f482ecfcde0011305d93fa6059f624a50...,8bf8f58e615549df5f05b717b6e914da80e213edab3dba...,chf,9.0,BASIC,1,2017-09-26,2022-10-08,False,60.387277,61.0,9.0,7.5,549.0,457.5
3,69d21e601612cc97fc7df9bd17a281879f785358d1c681...,cf914aa939499e8e18fdd9494fce5297b9dce2e1f909ea...,chf,90.0,BASIC,0,2017-09-28,2021-09-27,False,47.968131,48.0,90.0,75.0,360.0,300.0
4,29f98a3154a679f2fe953b1f3b148ddffcb28e60fff719...,8759ab0e44c1cf688770220cf1b87efac2726baca6bfcb...,eur,11.0,STANDARD,1,2017-10-13,2021-10-12,False,47.968131,48.0,15.0,12.5,720.0,600.0


In [66]:
#test
df_subscriptions_clean.query('gift_subscription == True')

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription,subscription_months_raw,subscription_months,price_chf,price_eur,total_price_chf,total_price_eur
129,1b249448b548340a4c331b798ca19d3d9e6b8c5bb2fe2b...,ca8d12609909bca935242f57ad6729743ee5bda540370d...,chf,NaN,BASIC,0,2019-10-09,2020-10-09,True,12.024888,12.0,90.0,75.0,90.0,75.0
152,d8121a15318743f48377b45118cd136d338a4b99168b50...,c6747adbc9d62f177c6df80d0512e02b273fbaf9af18e9...,chf,NaN,BASIC,0,2019-10-23,2020-10-23,True,12.024888,12.0,90.0,75.0,90.0,75.0
154,4e063374f5b5776d961dc6af23201f2f3b02a2e0e7c8ec...,bbf79862d11f54be917d779a62f927a31c9b608c54b897...,chf,NaN,STANDARD,0,2019-10-25,2020-10-25,True,12.024888,12.0,150.0,125.0,150.0,125.0
155,72e221592a53adf4b16d3377f0035a94476e3205b8de0c...,29e95e62d3aad00e06f4793f346c4460efd66f1d7e385a...,chf,NaN,STANDARD,0,2019-10-26,2020-10-26,True,12.024888,12.0,150.0,125.0,150.0,125.0
161,e72b4435f2edc79d2a6246ccfcf19c1d082bfcb12bcc2f...,7aa2b073f1bef9f54695ea8e0379bf7b49dee885127717...,chf,NaN,BASIC,0,2019-10-30,2020-10-30,True,12.024888,12.0,90.0,75.0,90.0,75.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12088,522e99f4fa00bf3c5c4fba995435da2e567c72f99b556c...,c98d77572029ad869635c9462f5bc3f3192f6629e06330...,chf,NaN,BASIC,0,2022-11-18,2023-05-17,True,5.913879,6.0,90.0,75.0,49,41
12092,8ac219a49c4ae53555b02f2f7feabcea1cfea745a60a0a...,e2e764388c142feb9a1845ea6a62324804dd41729f949a...,chf,NaN,BASIC,0,2022-11-19,2023-05-18,True,5.913879,6.0,90.0,75.0,49,41
12102,687766a0e33a1dd2382a44c955cdccb6d8576225ceb7b4...,b8fb027fd67f77f35fe45c6c20b24e3851ab6ffea6c5d7...,chf,NaN,BASIC,0,2022-11-20,2023-11-19,True,11.959178,12.0,90.0,75.0,90.0,75.0
12106,d6de7106e61d97730bd2d82a10890437339a9bf732550f...,447a4e9623204cc19827c425ae0c6616baaaa201c53e12...,chf,NaN,BASIC,0,2022-11-20,2023-05-19,True,5.913879,6.0,90.0,75.0,49,41


In [67]:
#test
df_subscriptions_clean.query('subscription_monthly == 1')

,subscription_key,account_key,currency,price,subscription_type,subscription_monthly,subscription_start,subscription_end,gift_subscription,subscription_months_raw,subscription_months,price_chf,price_eur,total_price_chf,total_price_eur
2,40f9e218b70a6f482ecfcde0011305d93fa6059f624a50...,8bf8f58e615549df5f05b717b6e914da80e213edab3dba...,chf,9.0,BASIC,1,2017-09-26,2022-10-08,False,60.387277,61.0,9.0,7.5,549.0,457.5
4,29f98a3154a679f2fe953b1f3b148ddffcb28e60fff719...,8759ab0e44c1cf688770220cf1b87efac2726baca6bfcb...,eur,11.0,STANDARD,1,2017-10-13,2021-10-12,False,47.968131,48.0,15.0,12.5,720.0,600.0
7,9f22270608209a507f93f3a74448a78a82f9741bc36ea3...,2c2621bf34f1cc419c0868f27ffece66dc1fb4f5ec665c...,chf,9.0,BASIC,1,2017-12-21,2020-11-08,False,34.596193,35.0,9.0,7.5,315.0,262.5
8,03411f3ad7d86f2e1df3db5a337c3025ba35159eb9b41e...,163db260041d30a58540a19312d416c9fc7cc696754108...,chf,9.0,BASIC,1,2017-12-22,2020-10-08,False,33.544837,34.0,9.0,7.5,306.0,255.0
12,3ad77a78e43113361666ec1a4b06540fc2ddd84ae7cbab...,3c84a8fb5508a115e73dc7d863452c4eb4e5ce7033adf6...,chf,9.0,BASIC,1,2018-02-08,2020-11-12,False,33.117723,34.0,9.0,7.5,306.0,255.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12099,70c55494a3f9eefcd90a671e8a74bccc312139353d4c7b...,7ec7fe79ff700191fb6798ae75436adfb4fc9ee8b9624c...,chf,9.0,BASIC,1,2022-11-19,2022-12-18,False,0.952792,1.0,9.0,7.5,9.0,7.5
12105,d748890de6ab3fe84dfca7f1d52f3427144500cfffd88c...,ccf2385b424a73c8e3935d5f6253f66031fa2dfff57b97...,chf,15.0,STANDARD,1,2022-11-20,2022-12-19,False,0.952792,1.0,15.0,12.5,15.0,12.5
12108,e155204feaa91e1c72695dd4a3277cffc1611002d5a78c...,8382d7280f045ff422da10d4cbae2e2011e69f76c1eced...,chf,9.0,BASIC,1,2022-11-20,2022-12-19,False,0.952792,1.0,9.0,7.5,9.0,7.5
12112,2b7087543ab0092df7c20c3acf0d2c2326952cd7dbebad...,e91c854aa33878f97f93b1b12adbff0526b91b94532c51...,chf,9.0,BASIC,1,2022-11-20,2022-12-19,False,0.952792,1.0,9.0,7.5,9.0,7.5


In [71]:
#display(gifted_df.groupby('subscription_months').max())
print(df_subscriptions_clean.groupby(['subscription_months','subscription_monthly']).sum())

                                            price  gift_subscription  \
subscription_months subscription_monthly                               
-0.0                0                      3745.0                  0   
                    1                       345.0                  0   
 1.0                0                      6810.0                  3   
                    1                     17161.5                  0   
 2.0                0                      2190.0                  1   
...                                           ...                ...   
 48.0               1                        11.0                  0   
 49.0               1                         9.0                  0   
 60.0               0                       660.0                  0   
 61.0               1                         9.0                  0   
 72.0               0                        90.0                  0   

                                          subscription_months_r

In [ ]:
df_subscriptions_clean.to_csv('./data/subcriptions_clean.csv')

In [ ]:
# gifted_df= df_subscriptions_clean.loc[df_subscriptions_clean['gift_subscription']==True]
# gifted_df['months'] = gifted_df['subscription_months'].round()
# gifted_df.tail(15)
# #display(gifted_df.groupby('subscription_months').max())
# gifted_df.groupby('months').sum()



In [ ]:
nongifted_df= df_subscriptions_clean.loc[df_subscriptions_clean['gift_subscription']==False]

nongifted_df.groupby(['months', 'subscription_monthly']).sum()

In [ ]:
#nongifted_df.query('account_key == b43a87d35bf285afdbb1c931b68ea2e6dad1f9dcc62947')

nongifted_df[nongifted_df['account_key'].str.contains('d8aa9f6793e94bc168a65808c9fe5809d4516448eae392a33edec16391c71d1e', na=False)]


In [ ]:
# df_subscriptions_clean = df_subscriptions_clean.drop('calc_price_chf', axis=1)

# Vouchers

In [ ]:
# set paths to csv files
accounts = './data/accounts.csv'
playbacks = './data/playbacks.csv'
subscriptions = './data/subscriptions.csv'
vouchers = './data/promo_vouchers.csv'

In [ ]:
# Read promo_couchers
df_vouchers = pd.read_csv(vouchers, sep=';')
# set column names to lowercase
df_vouchers.columns = df_vouchers.columns.str.lower()

In [ ]:
df_vouchers.head()

,movie_id,creationdate,expirationdate,account_key,voucherused,email_hash
0,476,2020-01-01 21:05:52,2020-01-31,68b8685cdb36f097528a01b41015c8a5f4766e1d5ef619...,1,338da97810604387739520be9bac3f8d4e686357e6e9ef...
1,319,2020-01-01 23:18:27,2020-01-31,4ef8153eea048568537a29b6429d65b35c30414b4ab753...,0,c19709fdf4628b8eb0746954b5edd57d1880cb17ac6bf8...
2,300,2020-01-02 22:44:00,2020-02-01,cbc315a9085a2ef1bf1fbe833069c87cb4b0913137ae8f...,0,22b0a84989d222bf683bc18ef6852f00de9f91d77f47ab...
3,586,2020-01-03 07:15:06,2020-02-02,3f4446380aadf0b86ee39820962d0d0c5824a782bed6b0...,1,88d1f0201ba08e0a578e66f236991c939ee01224654230...
4,585,2020-01-03 12:42:59,2020-02-02,6d45e9cb75dd7daaf7dddfb7c0b3fe762252954db15f63...,0,999c12ed0a20ed60440a91c2c57a236bd329859e8da19f...


In [ ]:
# changing datatype to datetime
df_vouchers['creationdate'] = pd.to_datetime(df_vouchers['creationdate'])
df_vouchers['expirationdate'] = pd.to_datetime(df_vouchers['expirationdate'])

In [ ]:
#cut off time of creation -> time not relevant? 
#df_vouchers['creationdate'] = df_vouchers['creationdate'].dt.date -> does not work very well as it returns an object
df_vouchers['creationdate'] = df_vouchers['creationdate'].dt.normalize()

In [ ]:
df_vouchers.dtypes

movie_id                   int64
creationdate      datetime64[ns]
expirationdate    datetime64[ns]
account_key               object
voucherused                int64
email_hash                object
dtype: object

In [75]:
df_vouchers.groupby(['email_hash']).count().sort_values(by='account_key', ascending=False)
# 14 times shared to same account - not in scope of promo campaign -> not a "new customer"

,movie_id,creationdate,expirationdate,account_key,voucherused
email_hash,,,,,
963ae58ad73365f111461a49367d4a699bbc978a4bfcb18d8fa1b3acc77dea96,14,14,14,14,14
8e89633bf847cd9232d3d414535f6a4a7cbe68573cfd250985ca4b973c3f6579,11,11,11,11,11
6c10e0aa2daa7da78fc07feded16c106cc69932d6bf1f0698f8c673ab1690bcf,10,10,10,10,10
9f605261d940cae39bb60d4c7c1a61bea3f4d716373f1a357df379548dffb41d,9,9,9,9,9
20468acea25b3e4eb1d9fea0ce62a38170f12d1b06a130f27d8e62e5314f13e9,9,9,9,9,9
...,...,...,...,...,...
5f3d3a3b50151afd509ced7ca5e5989499abc0bc6b13f18215b4fa3b6041562a,1,1,1,1,1
5f30d421846678037dd576548d8d576736b05aee6ef8bf65d8595527cfbeee3c,1,1,1,1,1
5f21bf5734d6b4f5ee98e0fd8ddf33ea64f6eb01d0a5fdb8b15a7e213aee776f,1,1,1,1,1


In [77]:
df_vouchers.groupby(['email_hash', 'account_key']).count().sort_values(by='movie_id', ascending=False)
#most emails on top get vouchers from same account_key 

,,movie_id,creationdate,expirationdate,voucherused
email_hash,account_key,,,,
963ae58ad73365f111461a49367d4a699bbc978a4bfcb18d8fa1b3acc77dea96,43169f236aa21371cb6698e42ecaf1a21c175aefff19899b6fc8893f2494aa36,14,14,14,14
6c10e0aa2daa7da78fc07feded16c106cc69932d6bf1f0698f8c673ab1690bcf,b75fd8923545ce8dc5281cdd97d5fbe78822cb1e72b1bbd5472c2201e27fe2d0,10,10,10,10
1b70f8ca6a294e99b2413d3bd33e99b0933e52917ee456f3621be8b7fc8ee0b7,76a1495f24017fbe9cad15a1002848d82084c106ce4e6cf7c33e2213b9ffac09,9,9,9,9
dbb93303e37cbe83e422167f3ba956777ff02dde1007f6ca070ca592214db56c,304567f00bcb6b2c342429554af05104e10c2de331ba0bbe266d7a79472a38e0,9,9,9,9
d85e3b1aee8bc1d4a7b22b3b27fd8b5adfb6d79c514677a836533fd734ac9ae1,13efc9cebdfedae76dfc3fe8a7b4779e207466010e47c2081d191f48d9ffcaf0,8,8,8,8
...,...,...,...,...,...
5eab4e395056459c76e11a19112ebb44b9b9943e8be11660ee2dd0a92f972773,de8731b64718d3430a41186b092ef392d684e8d09cba42782db811b8b3d1d74e,1,1,1,1
5ea6257a389574ee87f86c8c13ffb6636ff622f53b6eea96360c83906ad8be62,a9753738ba976b519f0c5f1f82dc9306732fa11cffbc994edf4a7ba6611779e7,1,1,1,1
5e8dae0f4806873bd6f32a5771f73b47513bd238b547de1e1ba1c96062f23879,271f4b45553d20371674d68169a95892418faf9b875c7b33b64c678efbd2305b,1,1,1,1


In [78]:
df_vouchers.groupby(['voucherused']).count().sort_values(by='movie_id', ascending=False)
# approx 50% do not redeem their voucher

,movie_id,creationdate,expirationdate,account_key,email_hash
voucherused,,,,,
0,3913,3913,3913,3913,3913
1,3869,3869,3869,3869,3869
